# ECG Analysis Pipeline Demo

This notebook demonstrates the complete ECG analysis pipeline for calculating heart rate and HRV metrics from uploaded ECG data.

## Features
- **Multi-format support**: CSV, TXT, WFDB formats
- **Signal preprocessing**: Bandpass filtering (0.5-40 Hz)
- **R-peak detection**: Multiple algorithms (NeuroKit2, SciPy, Pan-Tompkins)
- **Heart rate calculation**: Average BPM
- **HRV metrics**: SDNN, RMSSD, Mean RR, pNN50
- **Quality assessment**: Signal quality scoring
- **Visualization**: ECG plots with R-peaks and RR analysis

## 📦 Install Required Packages

In [ ]:
# Install required packages
!pip install numpy scipy matplotlib pandas
!pip install neurokit2  # For advanced ECG analysis
!pip install wfdb       # For WFDB format support

print("✅ All packages installed")

## 📥 Import Libraries and Load ECG Analyzer

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Import our ECG analyzer
import sys
sys.path.append('../src')  # Add src to path

from core.ecg_analyzer import ECGAnalyzer, quick_analysis

print("📊 ECG Analysis Pipeline Ready")
print("Upload your ECG data file to analyze!")

## 🔬 Method 1: Complete Analysis Pipeline

Use this method for detailed analysis with full control over parameters.

In [ ]:
# Initialize ECG Analyzer
analyzer = ECGAnalyzer(sampling_rate=None)  # Will auto-detect sampling rate

# Upload and analyze your ECG file
# Replace 'your_ecg_file.csv' with your actual file path
file_path = "your_ecg_file.csv"  # 👈 CHANGE THIS TO YOUR FILE PATH

try:
    # Run complete analysis
    results = analyzer.analyze(
        file_path=file_path,
        channel=0,    # Use first channel/lead
        plot=True     # Generate plots
    )
    
    print("\n🎉 Analysis completed successfully!")
    
except FileNotFoundError:
    print("❌ File not found. Please upload your ECG data file and update the file_path variable.")
    print("   Supported formats: CSV, TXT, WFDB (.dat/.hea)")
    
except Exception as e:
    print(f"❌ Analysis error: {str(e)}")

## ⚡ Method 2: Quick Analysis

Use this method for fast analysis with default settings.

In [ ]:
# Quick analysis - one line of code!
file_path = "your_ecg_file.csv"  # 👈 CHANGE THIS TO YOUR FILE PATH

try:
    results = quick_analysis(file_path)
    
    print("🚀 Quick Analysis Results:")
    for key, value in results.items():
        print(f"   {key}: {value:.2f}")
        
except Exception as e:
    print(f"❌ Error: {str(e)}")

## 🎛️ Step-by-Step Analysis (Advanced)

For detailed control over each analysis step.

In [ ]:
# Step-by-step analysis
analyzer = ECGAnalyzer()
file_path = "your_ecg_file.csv"  # 👈 CHANGE THIS

try:
    # Step 1: Load data
    print("Step 1: Loading ECG data...")
    signal = analyzer.load_data(file_path, format_type='auto', channel=0)
    
    # Step 2: Preprocess
    print("\nStep 2: Preprocessing signal...")
    filtered_signal = analyzer.preprocess(lowcut=0.5, highcut=40.0)
    
    # Step 3: Detect R-peaks
    print("\nStep 3: Detecting R-peaks...")
    r_peaks = analyzer.detect_r_peaks(method='neurokit')  # or 'scipy', 'pantompkins'
    
    # Step 4: Calculate heart rate
    print("\nStep 4: Calculating heart rate...")
    hr = analyzer.calculate_heart_rate()
    
    # Step 5: Calculate HRV metrics
    print("\nStep 5: Calculating HRV metrics...")
    hrv_metrics = analyzer.calculate_hrv_metrics()
    
    # Step 6: Assess signal quality
    print("\nStep 6: Assessing signal quality...")
    quality = analyzer.assess_signal_quality()
    
    # Step 7: Generate plots
    print("\nStep 7: Generating plots...")
    fig = analyzer.plot_results(figsize=(15, 12))
    plt.show()
    
except Exception as e:
    print(f"❌ Error in step-by-step analysis: {str(e)}")

## 📊 Detailed Results Analysis

In [ ]:
# Display detailed results if analysis was successful
if 'results' in locals() and analyzer.signal is not None:
    
    print("📈 DETAILED ECG ANALYSIS RESULTS")
    print("=" * 50)
    
    # Basic signal info
    duration = len(analyzer.signal) / analyzer.sampling_rate
    print(f"📊 Signal Information:")
    print(f"   • Duration: {duration:.1f} seconds")
    print(f"   • Sampling Rate: {analyzer.sampling_rate} Hz")
    print(f"   • Total Samples: {len(analyzer.signal):,}")
    print(f"   • Signal Range: {np.ptp(analyzer.signal):.3f} mV")
    
    # Heart rate results
    print(f"\n💓 Heart Rate Analysis:")
    print(f"   • Average HR: {results['heart_rate_bpm']:.1f} BPM")
    print(f"   • R-peaks detected: {len(analyzer.r_peaks)}")
    print(f"   • Beats per minute: {len(analyzer.r_peaks) / (duration/60):.1f}")
    
    # HRV results
    print(f"\n📊 HRV Time-Domain Metrics:")
    print(f"   • Mean RR: {results['mean_rr']:.1f} ms")
    print(f"   • SDNN: {results['sdnn']:.1f} ms (overall variability)")
    print(f"   • RMSSD: {results['rmssd']:.1f} ms (short-term variability)")
    print(f"   • pNN50: {results['pnn50']:.1f}% (adjacent intervals >50ms different)")
    
    # Quality assessment
    quality_text = "Excellent" if results['signal_quality'] > 0.8 else \
                  "Good" if results['signal_quality'] > 0.6 else \
                  "Fair" if results['signal_quality'] > 0.4 else "Poor"
    
    print(f"\n🎯 Quality Assessment:")
    print(f"   • Signal Quality: {quality_text} ({results['signal_quality']:.2f}/1.0)")
    
    # Clinical interpretation
    print(f"\n🏥 Clinical Context:")
    hr = results['heart_rate_bpm']
    if hr < 60:
        print(f"   • Bradycardia: Heart rate below 60 BPM")
    elif hr > 100:
        print(f"   • Tachycardia: Heart rate above 100 BPM")
    else:
        print(f"   • Normal heart rate: 60-100 BPM range")
    
    # HRV interpretation
    sdnn = results['sdnn']
    if sdnn < 50:
        print(f"   • Low HRV: SDNN < 50ms may indicate reduced autonomic function")
    elif sdnn > 50:
        print(f"   • Normal/High HRV: SDNN > 50ms indicates good autonomic function")
        
else:
    print("❌ No analysis results available. Please run the analysis first.")

## 📁 Batch Processing Multiple Files

In [ ]:
# Process multiple ECG files
import glob
from pathlib import Path

# Define your data directory
data_directory = "path/to/your/ecg/files/"  # 👈 CHANGE THIS

# Find all ECG files (adjust pattern as needed)
file_patterns = ['*.csv', '*.txt', '*.dat']
ecg_files = []

for pattern in file_patterns:
    ecg_files.extend(glob.glob(f"{data_directory}/{pattern}"))

print(f"Found {len(ecg_files)} ECG files")

# Process each file
batch_results = []

for file_path in ecg_files[:5]:  # Process first 5 files
    try:
        print(f"\n🔬 Processing: {Path(file_path).name}")
        
        result = quick_analysis(file_path)
        result['filename'] = Path(file_path).name
        batch_results.append(result)
        
        print(f"   HR: {result['heart_rate_bpm']:.1f} BPM, SDNN: {result['sdnn']:.1f} ms")
        
    except Exception as e:
        print(f"   ❌ Error: {str(e)}")

# Create summary DataFrame
if batch_results:
    df_results = pd.DataFrame(batch_results)
    print("\n📊 Batch Processing Summary:")
    print(df_results[['filename', 'heart_rate_bpm', 'sdnn', 'rmssd', 'signal_quality']].round(2))
else:
    print("No files processed successfully")

## 🛠️ Custom Parameters and Fine-tuning

In [ ]:
# Custom analysis with specific parameters
analyzer = ECGAnalyzer(sampling_rate=250)  # Specify sampling rate if known

file_path = "your_ecg_file.csv"  # 👈 CHANGE THIS

try:
    # Load with specific parameters
    analyzer.load_data(file_path, format_type='csv', channel=0)
    
    # Custom preprocessing
    analyzer.preprocess(
        lowcut=1.0,      # Higher low-cut frequency
        highcut=30.0,    # Lower high-cut frequency  
        filter_order=6   # Higher filter order
    )
    
    # Try different R-peak detection methods
    methods = ['neurokit', 'scipy', 'pantompkins']
    
    for method in methods:
        print(f"\n🔍 Testing {method} method:")
        try:
            peaks = analyzer.detect_r_peaks(method=method)
            hr = analyzer.calculate_heart_rate()
            print(f"   Found {len(peaks)} peaks, HR: {hr:.1f} BPM")
        except Exception as e:
            print(f"   ❌ {method} failed: {str(e)}")
    
    # Generate final results
    hrv = analyzer.calculate_hrv_metrics()
    quality = analyzer.assess_signal_quality()
    
    # Custom visualization
    fig = analyzer.plot_results(figsize=(20, 15))
    plt.show()
    
except Exception as e:
    print(f"❌ Custom analysis error: {str(e)}")

## 💾 Save Results to File

In [ ]:
# Save analysis results
if 'results' in locals():
    
    # Save to CSV
    results_df = pd.DataFrame([results])
    results_df.to_csv('ecg_analysis_results.csv', index=False)
    print("✅ Results saved to: ecg_analysis_results.csv")
    
    # Save detailed report
    with open('ecg_analysis_report.txt', 'w') as f:
        f.write("ECG ANALYSIS REPORT\n")
        f.write("=" * 50 + "\n\n")
        
        f.write(f"File: {file_path}\n")
        f.write(f"Analysis Date: {pd.Timestamp.now()}\n\n")
        
        f.write("HEART RATE ANALYSIS:\n")
        f.write(f"Average HR: {results['heart_rate_bpm']:.1f} BPM\n\n")
        
        f.write("HRV METRICS:\n")
        f.write(f"Mean RR: {results['mean_rr']:.1f} ms\n")
        f.write(f"SDNN: {results['sdnn']:.1f} ms\n")
        f.write(f"RMSSD: {results['rmssd']:.1f} ms\n")
        f.write(f"pNN50: {results['pnn50']:.1f}%\n\n")
        
        f.write(f"Signal Quality: {results['signal_quality']:.2f}\n")
    
    print("✅ Detailed report saved to: ecg_analysis_report.txt")
    
    # Save plot
    if 'fig' in locals():
        fig.savefig('ecg_analysis_plot.png', dpi=300, bbox_inches='tight')
        print("✅ Plot saved to: ecg_analysis_plot.png")
        
else:
    print("❌ No results to save. Run analysis first.")

## 📖 Usage Instructions

### Quick Start:
1. **Upload your ECG file** to the notebook environment
2. **Update the `file_path`** variable in any cell above
3. **Run the analysis** using Method 1 or Method 2

### Supported File Formats:
- **CSV files**: Comma-separated values with ECG signal data
- **TXT files**: Tab or space-separated text files
- **WFDB format**: PhysioNet database format (.dat + .hea files)

### Expected Data Format:
- **Single column**: One ECG signal per file
- **Multiple columns**: Specify channel (0 for first column)
- **Sampling rate**: Will be auto-detected or can be specified

### Output Metrics:
- **Heart Rate**: Average BPM
- **Mean RR**: Average time between heartbeats (ms)
- **SDNN**: Standard deviation of RR intervals (overall HRV)
- **RMSSD**: Root mean square of successive RR differences (short-term HRV)
- **pNN50**: Percentage of adjacent RR intervals differing by >50ms
- **Signal Quality**: 0-1 score indicating data quality

### Troubleshooting:
- **"File not found"**: Check file path and upload status
- **"No peaks detected"**: Signal may be inverted or very noisy
- **"Poor signal quality"**: Try different preprocessing parameters
- **Low peak count**: Check sampling rate and signal amplitude